In [87]:
import pandas as pd
from html import unescape
import json
import numpy as np
from tqdm.notebook import tqdm

In [88]:
true_tags = pd.read_csv("true-tags.csv")
true_tags["user_list"] = true_tags["User Tags"].apply(lambda r: r.split(";") if isinstance(r, str) else [])
true_tags["signage_list"] = true_tags["Signage Tags"].apply(lambda r: r.split(";") if isinstance(r, str) else [])
# true_tags.head(5)

In [89]:
with open("k-steps.json") as f:
    k_steps = json.load(f)

In [90]:
k_steps.keys()

dict_keys(['500', '675', '850', '1025', '1200'])

In [103]:
chosen_step = k_steps["850"]

In [104]:
cluster_1 = chosen_step[0]
cluster_1

['growth and development organization', 'growth and development organisation']

In [105]:
filtered_ids = list(true_tags[true_tags["user_list"].apply(lambda r: len(set(r) & set(cluster_1)) > 0)]["Image Id"])
len(filtered_ids)

7

In [106]:
first_filtered_id = filtered_ids[0]
first_filtered_id

3489

In [107]:
if "clip_probs" not in globals():
    with open("clip_probs_6dec.json") as f:
        clip_probs = json.load(f)

In [108]:
if "user_tags" not in globals():
    with open("../user_tags_set.json") as f:
        user_tags = np.array(json.load(f))
user_tags_dict = {t: i for i, t in enumerate(user_tags)}

In [109]:
user_tags_dict["sales and marketing"]

534

In [110]:
probabilities = {key: clip_probs[str(first_filtered_id)]["user"][user_tags_dict[key]] for key in cluster_1}
probabilities

{'growth and development organization': 0.000104,
 'growth and development organisation': 0.000216}

In [111]:
from collections import defaultdict
# TODO: Remove this from notebook
running_sums = defaultdict(int)
for id in filtered_ids:
    probabilities = {key: clip_probs[str(id)]["user"][user_tags_dict[key]] for key in cluster_1}
    for key, value in probabilities.items():
        running_sums[key] += value
for key, value in running_sums.items():
    running_sums[key] /= len(filtered_ids)

In [112]:
running_sums

defaultdict(int,
            {'growth and development organization': 0.0009952857142857144,
             'growth and development organisation': 0.0008651428571428571})

In [139]:
# Now do the same thing for each cluster

def top_probs_for_clusters(clusters):
    cluster_sums = []
    for cluster in tqdm(clusters):
        running_sums = defaultdict(int)
        filtered_ids = list(true_tags[true_tags["user_list"].apply(lambda r: len(set(r) & set(cluster)) > 0)]["Image Id"])
        for id in filtered_ids:
            probabilities = {key: clip_probs[str(id)]["user"][user_tags_dict[key]] for key in cluster}
            for key, value in probabilities.items():
                running_sums[key] += value
        for key, value in running_sums.items():
            running_sums[key] /= len(filtered_ids)
            running_sums[key] = round(running_sums[key], 4)
        # Sort running sums
        running_sums = dict(sorted(running_sums.items(), key=lambda x: x[1], reverse=True))
        cluster_sums.append(running_sums)
    return cluster_sums

In [140]:
k_top_probs = {
    k: top_probs_for_clusters(clusters) for k, clusters in tqdm(k_steps.items())
}


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/427 [00:00<?, ?it/s]

  0%|          | 0/409 [00:00<?, ?it/s]

  0%|          | 0/372 [00:00<?, ?it/s]

  0%|          | 0/319 [00:00<?, ?it/s]

In [144]:
k_top_probs["1200"][-5:]

[{'electronics manufacture': 0.0099,
  'electronics services': 0.0079,
  'semiconductor': 0.0039,
  'electronics': 0.0033},
 {'parking management': 0.0162,
  'parking': 0.0151,
  'parking lot': 0.0065,
  'park': 0.0035,
  'no parking': 0.0025},
 {'athletics': 0.0041,
  'sports': 0.0036,
  'community sports': 0.0029,
  'basketball': 0.0019,
  'tennis.': 0.0019,
  'athletes': 0.0018},
 {'club': 0.0135,
  'tennis club': 0.0104,
  'football club': 0.0101,
  'country club': 0.0046,
  'garden club': 0.0043,
  'wine club': 0.0035},
 {'church event': 0.0272,
  'non-denominational church': 0.0067,
  'church office': 0.0058,
  'church planning': 0.003,
  'church property': 0.0029,
  'church': 0.0021}]

In [142]:
with open("k-top-probs.json", "w") as f:
    json.dump(k_top_probs, f)